<a href="https://colab.research.google.com/github/Tm-ui/Exploratory-Data-Analysis-Company-Health./blob/main/Exploratory_Data_Analysis_Company_Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Cleaning- Followed these Steps: Reindex the Questions: Ensure the questions are indexed from 1 to 75 to match the number of responses.Question indexing starts at 3 and ends at 78. Separate the Responses: Split the string of responses from the responses CSV into individual columns, each representing a response to a specific question. Correlate Responses with Questions: Align the responses to each question chronologically. Begin Analysis: Once data alignment is ensured, we proceed with the analysis.

In [ ]:
import pandas as pd
import uuid

# 1. First, reindex the questions from 1 to 75 in the questions DataFrame.

# Step 1: Load and Clean the Responses DataFrame

# Load the responses DataFrame (anonymized file path)
responses_df = pd.read_csv('/content/responses.csv')

# List of valid responders (anonymized responder names)
valid_responders = ['User_1', 'User_2', 'User_3', 'User_4', 'User_5', 'User_6']

# Filter the DataFrame to keep only rows with valid responders
cleaned_responses_df = responses_df[responses_df['ResponderName'].isin(valid_responders)]

# Further filter out entries where 'ResponderName' is 'User_2' and 'Response' is empty
cleaned_responses_df = cleaned_responses_df[~((cleaned_responses_df['ResponderName'] == 'User_2') & (cleaned_responses_df['Response'].isna()))]

# Reset the index after filtering
cleaned_responses_df.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Cleaned Responses DataFrame:")
print(cleaned_responses_df.head())

# Step 2: Load and Reindex the Questions DataFrame

# Load the questions DataFrame (anonymized file path)
questions_df = pd.read_csv('/content/questions.csv')

# Ensure questions are indexed from 1 to 76
questions_df['question_index'] = range(1, 77)  # Corrected range for indexing

# Display to check the reindexing
print("\nReindexed Questions DataFrame:")
print(questions_df[['question_index', 'q_text', 'q_khi']].head())

# Step 3: Separate Responses into Individual Columns

# Separate the responses string into individual responses
responses_df_expanded = cleaned_responses_df['Response'].apply(lambda x: pd.Series(list(str(x)))).rename(columns=lambda x: f'Q{x+1}')

# Anonymize email and timestamp columns
cleaned_responses_df['ResponderEmail'] = 'email_redacted@example.com'
cleaned_responses_df['Creation Date'] = 'date_redacted'
cleaned_responses_df['Modified Date'] = 'date_redacted'

# Combine with responder details
combined_df = pd.concat([cleaned_responses_df[['ResponderName', 'ResponderEmail', 'Creation Date', 'Modified Date']], responses_df_expanded], axis=1)

# Display to check the combined DataFrame
print("\nCombined DataFrame with Separated Responses:")
print(combined_df.head())

# Step 4: Align Responses with Questions

# Ensure correct alignment by renaming response columns to include question text
response_columns = [col for col in combined_df.columns if col.startswith('Q')]
question_texts = questions_df.set_index('question_index')['q_text'].to_dict()

# Rename response columns to include question text
combined_df.rename(columns={f'Q{i}': f'Q{i}: {question_texts[i]}' for i in range(1, 76)}, inplace=True)

# Display the aligned DataFrame
print("\nAligned DataFrame with Question Texts:")
print(combined_df.head())


In [ ]:

# Save the anonymized combined DataFrame to CSV
anonymized_combined_save_path = '/content/anonymized_combined_df.csv'
combined_df.to_csv(anonymized_combined_save_path, index=False)

# Save the anonymized cleaned responses DataFrame to CSV
anonymized_responses_save_path = '/content/anonymized_cleaned_responses_df.csv'
cleaned_responses_df.to_csv(anonymized_responses_save_path, index=False)

# Notify user that the anonymized files have been saved
print(f"Anonymized Combined DataFrame saved to: {anonymized_combined_save_path}")
print(f"Anonymized Cleaned Responses DataFrame saved to: {anonymized_responses_save_path}")

# Step 5: Calculating the number of non-missing responses for each user
user_response_counts = cleaned_responses_df['Response'].apply(lambda x: len([char for char in str(x) if char not in [None, 'NaN', 'nan']]))

# Calculating the average number of responses per user
average_responses_per_user = user_response_counts.mean()

# Display the calculated average
print(f"Average number of responses per user: {average_responses_per_user}")


Data Analysis are based on these questions:

 Which question got the most positive, negative, or maybe responses?
Which category got the most positive, negative, or maybe responses?
Which user gave the most positive, negative, or maybe responses?

By following these steps:

Calculate the Counts of Each Response Type (0, 1, ?) for Each Question:

Determine which question has the highest counts for each response type.
Calculate the Counts of Each Response Type for Each Category:

Group the responses by category and calculate the counts of each response type.
Calculate the Counts of Each Response Type for Each User:

Group the responses by user and calculate the counts of each response time

In [ ]:

import matplotlib.pyplot as plt

# Step 6: Convert response columns to strings to avoid mixed type issues
response_columns = [col for col in combined_df.columns if 'Q' in col]
combined_df[response_columns] = combined_df[response_columns].astype(str)

# Exclude Q76 from analysis since it has missing values for most users
response_columns = [col for col in response_columns if col != 'Q76']

# Step 7: Check for missing values in response columns after excluding Q76
missing_values = combined_df[response_columns].isna().sum()
print("\nMissing Values in Response Columns After Excluding Q76:")
print(missing_values[missing_values > 0])

# Step 8: Recalculate the distribution of response values ('0', '1', '?')
response_counts = combined_df[response_columns].apply(pd.Series.value_counts).fillna(0)
print("\nResponse Value Counts:")
print(response_counts)

# Step 9: Visualize response distribution by question
plt.figure(figsize=(12, 8))
response_counts.T.plot(kind='bar', stacked=True)
plt.title('Response Distribution Across Questions')
plt.xlabel('Questions')
plt.ylabel('Count of Responses')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Step 10: Identify the question with most positive, negative, and maybe responses
most_positive_question = response_counts.loc['1'].idxmax()
most_negative_question = response_counts.loc['0'].idxmax()
most_maybe_question = response_counts.loc['?'].idxmax()

# Display the questions with the highest response counts for each category
print(f"Question with most positive responses: {most_positive_question}")
print(f"Question with most negative responses: {most_negative_question}")
print(f"Question with most 'maybe' responses: {most_maybe_question}")


In [ ]:
# Step 11: Perform analysis on the aligned DataFrame (Q1 to Q75)

# Calculate counts for each response type ('1', '0', and '?') for each question
positive_counts = combined_df[response_columns].apply(lambda x: (x == '1').sum(), axis=0)
negative_counts = combined_df[response_columns].apply(lambda x: (x == '0').sum(), axis=0)
maybe_counts = combined_df[response_columns].apply(lambda x: (x == '?').sum(), axis=0)

# Display the questions with the highest count of each response type
print(f"Question with most positive responses: {positive_counts.idxmax()}")
print(f"Question with most negative responses: {negative_counts.idxmax()}")
print(f"Question with most 'maybe' responses: {maybe_counts.idxmax()}")


Changed the color and sizes to clearly see the visualized correlations. (red for no, green for yes and grey for question mark(maybe)) Mixed Data Types: Some columns have mixed types (int64 and object), leading to errors during operations like sorting or counting. Missing Data: The last question (Q76) has missing values (NaN) for most users, and is excluded from analysis.

In [ ]:


# Step 12: Convert response columns to strings to avoid mixed type issues
combined_df[response_columns] = combined_df[response_columns].astype(str)

# Calculate the distribution of response values ('0', '1', '?')
response_counts = combined_df[response_columns].apply(pd.Series.value_counts).fillna(0)

# Define colors for the responses ('0' -> Red, '1' -> Green, '?' -> Grey)
colors = {'0': 'red', '1': 'green', '?': 'grey'}

# Step 13: Plot the response distribution with custom colors
plt.figure(figsize=(12, 8))
ax = response_counts.T.plot(kind='bar', stacked=True, color=[colors.get(x, '#333333') for x in response_counts.index])

# Set plot titles and labels
plt.title('Response Distribution Across Questions')
plt.xlabel('Questions')
plt.ylabel('Count of Responses')
plt.xticks(rotation=90)

# Add a legend for better clarity
plt.legend(title='Responses', labels=['Negative (0)', 'Positive (1)', 'Maybe (?)'])

# Save the plot as an image file
output_image_path = '/content/response_distribution_questions.png'
plt.savefig(output_image_path)

# Display the plot
plt.tight_layout()
plt.show()

# Notify the user that the plot has been saved
print(f"Plot saved as an image at: {output_image_path}")


In [ ]:
# Step 14: Load the questions DataFrame and map questions to categories
questions_df = pd.read_csv('/content/questions.csv')
questions_df['question_index'] = range(1, 77)  # Ensure correct question indexing

# Create a category map from the questions DataFrame
category_map = questions_df.set_index('question_index')['q_khi'].to_dict()

# Create a DataFrame to map responses to categories
response_category_df = pd.DataFrame(index=response_columns)

# Step 15: Map each question to its corresponding category
response_category_df['Category'] = response_category_df.index.map(lambda x: category_map[int(x.split(':')[0][1:])])

# Calculate the distribution of response values ('0', '1', '?') for each question
response_counts = combined_df[response_columns].apply(pd.Series.value_counts).fillna(0)

# Assign categories to the response counts DataFrame
response_counts = response_counts.T
response_counts['Category'] = response_counts.index.map(lambda x: category_map[int(x.split(':')[0][1:])])

# Step 16: Visualize the response distribution grouped by categories
categories = response_counts['Category'].unique()

# Define custom colors for response types
colors = {'0': 'red', '1': 'green', '?': 'grey'}

# Create subplots for each category
fig, axes = plt.subplots(nrows=len(categories), figsize=(14, 4 * len(categories)), sharex=True)

for ax, category in zip(axes, categories):
    # Filter data for the current category
    category_data = response_counts[response_counts['Category'] == category].drop(columns='Category')

    # Plot the response distribution for the current category using custom colors
    category_data.plot(kind='bar', stacked=True, color=[colors.get(x, '#333333') for x in category_data.columns], ax=ax)

    ax.set_title(f'Response Distribution for {category}')
    ax.set_xlabel('Questions')
    ax.set_ylabel('Count of Responses')
    ax.legend(title='Responses', labels=['Negative (0)', 'Positive (1)', 'Maybe (?)'])

# Step 17: Adjust the layout and save the plot as an image file
plt.tight_layout()

# Save the plot as an image for future reference
output_image_path_category = '/content/response_distribution_categories(Khi).png'
plt.savefig(output_image_path_category)

# Display the plot
plt.show()

# Notify the user where the image is saved
print(f"Plot of response distribution by categories saved as: {output_image_path_category}")


In [ ]:
# Step 18: Analyze and visualize response distributions by user

# Convert response columns to strings to avoid mixed type issues
response_columns = [col for col in combined_df.columns if 'Q' in col]
combined_df[response_columns] = combined_df[response_columns].astype(str)

# Exclude Q76 from analysis since it has missing values for most users
response_columns = [col for col in response_columns if col != 'Q76']

# Define colors for the responses
colors = {'0': 'red', '1': 'green', '?': 'grey'}

# Get unique user names from the combined DataFrame
users = combined_df['ResponderName'].unique()

# Step 19: Create subplots for each user to visualize their response distribution
fig, axes = plt.subplots(nrows=len(users), figsize=(14, 4 * len(users)), sharex=True)

for ax, user in zip(axes, users):
    # Filter responses for the current user
    user_responses = combined_df[combined_df['ResponderName'] == user][response_columns]

    # Calculate response counts for the user
    user_counts = user_responses.apply(pd.Series.value_counts).fillna(0).T

    # Plot the response distribution for the current user with custom colors
    user_counts.plot(kind='bar', stacked=True, color=[colors.get(x, '#333333') for x in user_counts.columns], ax=ax)

    ax.set_title(f'Response Distribution for {user}')
    ax.set_xlabel('Questions')
    ax.set_ylabel('Count of Responses')
    ax.legend(title='Responses', labels=['Negative (0)', 'Positive (1)', 'Maybe (?)'])

# Step 20: Adjust the layout and save the plot as an image file
plt.tight_layout()

# Save the plot as an image
output_image_path_users = '/content/response_distribution_users.png'
plt.savefig(output_image_path_users)

# Display the plot
plt.show()

# Notify the user where the image is saved
print(f"Plot of response distribution by users saved as: {output_image_path_users}")


In [ ]:
# Step 21: Visualize the response distribution for each user

# Convert response columns to strings to avoid mixed type issues
response_columns = [col for col in combined_df.columns if 'Q' in col]
combined_df[response_columns] = combined_df[response_columns].astype(str)

# Exclude Q76 from analysis since it has missing values for most users
response_columns = [col for col in response_columns if col != 'Q76']

# Define colors for the response types
colors = {'0': 'red', '1': 'green', '?': 'grey'}

# Get the unique responder names from the DataFrame
users = combined_df['ResponderName'].unique()

# Step 22: Create subplots to visualize each user's response distribution
fig, axes = plt.subplots(nrows=len(users), figsize=(14, 4 * len(users)), sharex=True)

# Iterate over each user to generate a separate plot
for ax, user in zip(axes, users):
    # Filter the responses for the current user
    user_responses = combined_df[combined_df['ResponderName'] == user][response_columns]

    # Calculate response counts for each question (0, 1, ?)
    user_counts = user_responses.apply(pd.Series.value_counts).fillna(0).T

    # Plot the response distribution for the current user
    user_counts.plot(kind='bar', stacked=True,
                     color=[colors.get(x, '#333333') for x in user_counts.columns], ax=ax)

    # Set titles and labels
    ax.set_title(f'Response Distribution for {user}')
    ax.set_xlabel('Questions')
    ax.set_ylabel('Count of Responses')
    ax.legend(title='Responses', labels=['Negative (0)', 'Positive (1)', 'Maybe (?)'])

# Step 23: Layout adjustments and saving the plot
plt.tight_layout()

# Save the user response distribution plot as an image
output_image_path_users = '/content/response_distribution_users.png'
plt.savefig(output_image_path_users)

# Display the plot in the notebook
plt.show()

# Notify where the plot image has been saved
print(f"Response distribution plot by user saved at: {output_image_path_users}")


In [ ]:
# Step 24: Map questions to their respective categories using the questions DataFrame
questions_df = pd.read_csv('/content/questions.csv.csv')

# Ensure the questions are indexed correctly and map them to categories
questions_df['question_index'] = range(1, 77)
category_map = questions_df.set_index('question_index')['q_khi'].to_dict()

# Step 25: Calculate response counts and assign categories
# Count occurrences of each response value ('0', '1', '?') for every question
response_counts = combined_df[response_columns].apply(pd.Series.value_counts).fillna(0).T

# Map each question to its category based on 'q_khi' and add as a column in response_counts
response_counts['Category'] = response_counts.index.map(lambda x: category_map[int(x.split(':')[0][1:])])

# Step 26: Aggregate the response counts by category
category_totals = response_counts.groupby('Category').sum()

# Step 27: Generate pie charts for each category's response distribution
for category in category_totals.index:
    # Create a pie chart for each category
    plt.figure(figsize=(6, 6))
    plt.pie(
        category_totals.loc[category],
        labels=['Negative (0)', 'Positive (1)', 'Maybe (?)'],
        autopct='%1.1f%%',
        colors=['red', 'green', 'grey'],
        startangle=140
    )
    plt.title(f'Response Distribution for {category}')
    plt.axis('equal')  # Ensures the pie chart is a perfect circle

    # Display the pie chart
    plt.show()

    # Save the pie chart to a file
    output_image_path_pie = f'/content/response_distribution_{category}_pie_chart.png'
    plt.savefig(output_image_path_pie)

    # Notify where the pie chart has been saved
    print(f"Pie chart for {category} saved at: {output_image_path_pie}")


In [ ]:
# Step 28: Convert data from the JSON-like structure into a DataFrame for easier manipulation
data = [
    {"ResponderName": "Theodor", "ResponderEmail": "", "Creation Date": "Aug 20, 2024 3:45 am", "Modified Date": "Aug 20, 2024 3:46 am",
    "Q1: I understand how my work contributes to Hovedkvarteret's overall strategy.": "1",
    "Q2: Hovedkvarteret's strategy is clearly communicated and consistently reinforced.": "1",
    "Q3: I believe Hovedkvarteret's strategy positions us well in the tech consulting market.": "0",
    "Q4: nan": "0", "Q5: nan": "0", "Q6: nan": "?", "Q7: nan": "?", "Q8: nan": "?",
    "Q9: nan": "1", "Q10: nan": "1", "Q11: nan": "1", "Q12: Hovedkvarteret can quickly adapt to changes in the tech consulting market.": "1",
    "Q13: We effectively balance current client needs with long-term innovation.": "1",
    # More responses go here...
    }
]

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Step 29: Select only columns that start with 'Q' as response columns
response_columns = [col for col in df.columns if col.startswith('Q')]

# Flatten the response columns into a single series for counting occurrences
all_responses = df[response_columns].stack()

# Count occurrences of each response type ('1', '0', '?')
response_counts = all_responses.value_counts()

# Step 30: Prepare data for the pie chart
labels = ['Positive (1)', 'Negative (0)', 'Maybe (?)']
sizes = [response_counts.get('1', 0), response_counts.get('0', 0), response_counts.get('?', 0)]
colors = ['green', 'red', 'grey']

# Step 31: Plot the pie chart for overall response distribution
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=140)
plt.title('Overall Distribution of Positive, Negative, and Maybe Responses')
plt.axis('equal')  # Ensures that the pie chart is a circle
plt.show()

# Step 32: Save the pie chart as an image file
plt.savefig('Overall Distribution Pie.png')

# Notify where the file is saved
print("Overall distribution pie chart saved as 'Overall Distribution Pie.png'")


In [ ]:

import numpy as np

# Step 1: Load the questions DataFrame and map questions to categories
questions_df = pd.read_csv('/content/questions.csv.csv')
questions_df['question_index'] = range(1, 77)
category_map = questions_df.set_index('question_index')['q_khi'].to_dict()

# Step 2: Extract response columns from the combined DataFrame
response_columns = [col for col in combined_df.columns if 'Q' in col]

# Step 3: Exclude Q76 from analysis if it has missing values for most users
response_columns = [col for col in response_columns if col != 'Q76']

# Step 4: Calculate the distribution of response values ('0', '1', '?') for each question
response_counts = combined_df[response_columns].apply(pd.Series.value_counts).fillna(0).T

# Step 5: Assign categories to each question based on the question index
response_counts['Category'] = response_counts.index.map(lambda x: category_map[int(x.split(':')[0][1:])])

# Step 6: Define colors for the responses
colors = {'0': 'red', '1': 'green', '?': 'grey'}

# Step 7: Iterate over each category to create a separate bar chart for each
categories = response_counts['Category'].unique()

for category in categories:
    # Filter data for the current category
    category_data = response_counts[response_counts['Category'] == category]

    # Prepare the data for plotting
    n_questions = len(category_data)  # Number of questions in the category
    bar_width = 0.2  # Width of each bar
    index = np.arange(n_questions)  # Index positions for questions

    # Initialize the plot for the current category
    fig, ax = plt.subplots(figsize=(14, 8))  # Adjusted figure size for clarity

    # Plot each response type within the current category
    for i, response_type in enumerate(['0', '1', '?']):
        ax.bar(index + i * bar_width, category_data[response_type], bar_width,
               label=response_type, color=colors[response_type])

    # Customize the x-axis to show question keys
    ax.set_xticks(index + bar_width)
    ax.set_xticklabels([f'Q{i+1}' for i in range(n_questions)], rotation=0, ha='center')
    ax.set_xlabel('Questions (Keys)')
    ax.set_ylabel('Count of Responses')
    ax.set_title(f'Response Distribution for {category}')

    # Adjust y-axis and remove default ticks for clarity
    ax.legend(title='Response Type')

    # Prepare full question texts for display under the plot
    full_question_texts = [questions_df.loc[int(q.split(':')[0][1:])-1, 'q_text'] for q in category_data.index]

    # Display full questions under the plot
    plt.figtext(0.5, -0.25, '\n'.join([f"{f'Q{i+1}'}: {text}" for i, text in enumerate(full_question_texts)]),
                wrap=True, horizontalalignment='center', fontsize=10, ha='center')

    # Save the plot as an image
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.35)  # Adjust bottom margin for question texts
    plt.savefig(f'response_distribution_{category}.png', bbox_inches='tight')  # Ensures everything is included in the saved file
    plt.show()


In [ ]:


# Step 1: Load the questions DataFrame and map questions to categories
questions_df = pd.read_csv('/content/questions.csv')
questions_df['question_index'] = range(1, 77)
category_map = questions_df.set_index('question_index')['q_khi'].to_dict()

# Step 2: Extract response columns from the combined DataFrame
response_columns = [col for col in combined_df.columns if 'Q' in col]

# Step 3: Exclude Q76 from analysis if it has missing values for most users
response_columns = [col for col in response_columns if col != 'Q76']

# Step 4: Calculate the distribution of response values ('0', '1', '?') for each question
response_counts = combined_df[response_columns].apply(pd.Series.value_counts).fillna(0).T

# Step 5: Assign categories to each question based on the question index
response_counts['Category'] = response_counts.index.map(lambda x: category_map[int(x.split(':')[0][1:])])

# Step 6: Define colors for the responses
colors = {'0': 'red', '1': 'green', '?': 'grey'}

# Step 7: Iterate over each category to create a separate bar chart for each
categories = response_counts['Category'].unique()

for category in categories:
    # Filter data for the current category
    category_data = response_counts[response_counts['Category'] == category]

    # Prepare the data for plotting
    n_questions = len(category_data)  # Number of questions in the category
    bar_width = 0.2  # Width of each bar
    index = np.arange(n_questions)  # Index positions for questions

    # Initialize the plot for the current category
    fig, ax = plt.subplots(figsize=(14, 8))  # Adjusted figure size for clarity

    # Plot each response type within the current category
    for i, response_type in enumerate(['0', '1', '?']):
        ax.bar(index + i * bar_width, category_data[response_type], bar_width,
               label=response_type, color=colors[response_type])

    # Customize the x-axis to show question keys
    ax.set_xticks(index + bar_width)
    ax.set_xticklabels([f'Q{i+1}' for i in range(n_questions)], rotation=0, ha='center')
    ax.set_xlabel('Questions (Keys)')
    ax.set_ylabel('Count of Responses')
    ax.set_title(f'Response Distribution for {category}')

    # Adjust y-axis and remove default ticks for clarity
    ax.legend(title='Response Type')

    # Prepare full question texts for display under the plot
    full_question_texts = [questions_df.loc[int(q.split(':')[0][1:])-1, 'q_text'] for q in category_data.index]

    # Display full questions under the plot
    plt.figtext(0.5, -0.25, '\n'.join([f"{f'Q{i+1}: '}{full_question_texts[i]}" for i in range(n_questions)]), wrap=True, horizontalalignment='center', fontsize=12)

    # Adjust layout and show plot
    plt.tight_layout()
    plt.show()
